In [1]:
import pandas as pd
import numpy as np

1. fare i residuals regression on factors
2. Verificare mean reversion dei residual.


## Variables:
* Stock ($i$) returns: $R_{i, t}, \quad i=1, \dots, N$ or $R_t \in \mathbb{R}^N$
* Factor ($j$) returns: $f_{j, t}, \quad j=1, \dots, p$ or $f_t \in \mathbb{R}^p$
* Exposures: $X_t \in \mathbb{R}^{N\times p}$ or $X_i \in \mathbb{R}^p$, $i = 1, \dots, N$

At each $t=1, \dots, T$,
$$
R_i = f^TX_i + \tilde{R_i} \quad \forall i=1, \dots, N
$$
Or:
$$
R = Xf + \tilde{R}
$$
Get
$$
\begin{aligned}
\hat{f_t} &= (X^TD^{-1}X)^{-1}X^TD^{-1}R_i \\
\hat{\tilde{R_i}} &= R - X\hat{f_t}
\end{aligned}
$$
And we have:
$$
X^T\hat{\tilde{R_i}} = 0 \in \mathbb{R}^p
$$